## Import libraries

## Import libraries

In [43]:
import pandas as pd
import requests
import string
string.punctuation
import re
import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import numpy as np
from datetime import timezone
import datetime
import time
import pandas as pd
import requests
import string
string.punctuation
import re
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from bs4 import BeautifulSoup
import contractions

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from nltk import word_tokenize   
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
import matplotlib.dates as mdate



from sklearn.preprocessing import (
    StandardScaler,
    PolynomialFeatures
)
from datetime import timezone
import datetime

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Create Function to retrieve dataset from reddit

Start from 1st April 2021 and later. Take from PTSD subreddit --> time_from = 1617235200

In [46]:
# Create function that takes in search name and limit for search and time

def get_data(title_name, data_min, column, time_from = 1617235200):
    # initiate date to search
    date = time_from

    #instatiate main df for PTSD
    df = pd.DataFrame(columns = column)


    while df.shape[0]<data_min:
        url = 'https://api.pushshift.io/reddit/search/submission'
        params ={
            'subreddit': title_name,
            'before': date,
            'size': 50
        }
        # create temporary df 
        data_temp = pd.DataFrame(requests.get(url,params).json()['data'])[column]

        # create list for index with [removed], '', [deleted] and NAN values
        index_df = list(
            data_temp[
                (data_temp['selftext'] == '[removed]')
                | (data_temp['selftext'] == '')
                | (data_temp['selftext'] == '[deleted]')
                | (data_temp['selftext'].isnull())
                
            ].index)

        # drop down all in index list 
        data_temp = data_temp.drop(index = index_df)
        data_temp.drop_duplicates(subset=['selftext', 'title'], keep='last', inplace=True)
        # concat with main df
        df = pd.concat([df,data_temp])

        # print to check on shape 
        print(df.shape)

        # set up new date for searching after 
        date = df.iloc[-1,-1]
        
    return df

To draw out CPTSD subreddit

In [47]:
col = ['author', 'subreddit', 'selftext', 'title', 'created_utc']

# creating df for PTSD
df_ptsd = get_data('CPTSD', 10000, col)

# Create another column for datetime 
df_ptsd['datetime'] = df_ptsd['created_utc'].apply(lambda x: pd.to_datetime(x, unit='s'))

(40, 5)
(82, 5)
(123, 5)
(165, 5)
(206, 5)
(252, 5)
(299, 5)
(341, 5)
(384, 5)
(429, 5)
(472, 5)
(521, 5)
(569, 5)
(615, 5)
(664, 5)
(709, 5)
(752, 5)
(799, 5)
(845, 5)
(893, 5)
(940, 5)
(990, 5)
(1037, 5)
(1084, 5)
(1134, 5)
(1183, 5)
(1231, 5)
(1275, 5)
(1323, 5)
(1373, 5)
(1422, 5)
(1471, 5)
(1519, 5)
(1569, 5)
(1617, 5)
(1665, 5)
(1712, 5)
(1752, 5)
(1787, 5)
(1831, 5)
(1869, 5)
(1913, 5)
(1949, 5)
(1991, 5)
(2040, 5)
(2080, 5)
(2117, 5)
(2159, 5)
(2203, 5)
(2244, 5)
(2283, 5)
(2320, 5)
(2363, 5)
(2406, 5)
(2448, 5)
(2492, 5)
(2528, 5)
(2565, 5)
(2605, 5)
(2650, 5)
(2699, 5)
(2747, 5)
(2797, 5)
(2847, 5)
(2895, 5)
(2942, 5)
(2991, 5)
(3039, 5)
(3087, 5)
(3135, 5)
(3182, 5)
(3230, 5)
(3277, 5)
(3326, 5)
(3375, 5)
(3422, 5)
(3466, 5)
(3513, 5)
(3559, 5)
(3605, 5)
(3652, 5)
(3700, 5)
(3749, 5)
(3796, 5)
(3843, 5)
(3892, 5)
(3940, 5)
(3985, 5)
(4034, 5)
(4082, 5)
(4130, 5)
(4174, 5)
(4222, 5)
(4269, 5)
(4315, 5)
(4362, 5)
(4408, 5)
(4456, 5)
(4494, 5)
(4537, 5)
(4580, 5)
(4619, 5)
(465

## Pre-Processing

Combining selftext and title into alltext

In [ ]:
# create all text column for self text and title combined 

df_ptsd['alltext'] = df_ptsd['selftext'] + ' ' + df_ptsd['title']

Calculate length of text and title

In [ ]:
# to create calculate length of text and title 

df_ptsd['length_text'] = df_ptsd['alltext'].map(lambda x: len(x))

Word count per post

In [ ]:
# to create calculate status word cout of text and title 
tokenizer = RegexpTokenizer('\s+', gaps=True)

def tokenizer_func(text):
    return len(tokenizer.tokenize(text))

df_ptsd['wrdcount_text'] = df_ptsd['alltext'].apply(lambda x: tokenizer_func(x))

Check for null values

In [ ]:
# found NAN coming from the link filter
df_ptsd.isnull().sum()

author           0
subreddit        0
selftext         0
title            0
created_utc      0
datetime         0
alltext          0
length_text      0
wrdcount_text    0
dtype: int64

I will now **create the target variable** to be used in classification modelling. From this point forward in all classification models, the **positive class (1) will be *r/PTSD*** and the **negative class (0) will be *r/Anxiety***.

In [ ]:
# change Ptsd to 1 and Anxiety to 0

df_ptsd['subreddit'] = df_ptsd['subreddit'].map(lambda x: 1 if x == 'ptsd' else 0)

Create datetime object into another column

In [ ]:
# Converting 'created_utc' column from epoch time to datetime object.

df_ptsd['datetime'] = df_ptsd['created_utc'].map(lambda ts: datetime.datetime.fromtimestamp(ts))

In [ ]:
def change_month(month):
    convert = month.to_pydatetime().month
    month_dict = {
        1: 'Jan',
        2: 'Feb',
        3: 'Mar',
        4: 'April',
        5: 'May',
        6: 'June',
        7: 'July',
        8: "Aug",
        9: 'Sept',
        10: 'Oct',
        11: 'Nov',
        12: 'Dec'
    }

    return month_dict[convert]

df_ptsd['month'] = df_ptsd['datetime'].apply(lambda x: x.to_pydatetime().month)

In [ ]:
df_ptsd['day'] = df_ptsd['datetime'].apply(lambda x: x.strftime('%A'))

### Cleaning text - Without Lemmatizing/stemming

I will be defining a function that helps to do the following: 

- **Remove HTML artefacts** using *BeautifulSoup* library.
- **Expand all contractions** using *contractions* library.
- **Remove all numbers, punctuations and special characters, except '-'** to keep hyphenated words, using *re* library.
- **Convert all text to lowercase**.
- **Tokenize all words (hyphenated words stay hyphenated)** using *RegexpTokenizer* from *nltk.tokenize*.
- **Remove all stopwords** using the english stopwords list from *nltk.corpus*.
- **Remove subreddit names** *('PTSD', 'Anxiety')* and other related words to avoid **target leakage**.
- **Join all tokenized words** into a string separated by spaces.

Stemming/Lemmatizing of words will be done during error analysis. 

In [ ]:
# Defining a function to clean all_text column following the steps mentioned above.
def clean_text(text_raw):
    
    # Removing any and all HTML artefacts from the text (just in case reddit API still left behind some).
    text_html_removed = BeautifulSoup(text_raw).get_text()
    
    # Expanding all contractions using the contractions library (for example isn't is converted to is not).
    text_contrac_fixed = contractions.fix(text_html_removed)
    
    # Using regex to remove all numbers, punctuations and special characters (except '-' to keep hyphenated words).
    text_punc_removed = re.sub(r'[^A-Za-z\-\s]', ' ', text_contrac_fixed)
    
    # Tokenizing the text after converting it to lowercase.
    # Regex here tokenizes by every character except one or more ('+') consecutive word characters ('\w') or hyphens ('-').
    text_tokens_list = RegexpTokenizer(r'[\w\-]+').tokenize(text_punc_removed.lower())
    
    # Removing stopwords using the english stopwords list from nltk.corpus.
    text_stopwords_removed = [word for word in text_tokens_list if word not in set(nltk.corpus.stopwords.words('english'))]
    
    # Removing subreddit name from text (target leakage).
    text_cleaned = [word for word in text_stopwords_removed if word not in set(['ptsd', 'anxious', 'anxiety',"cptsd", 'trauma','traumatized','traumas',"posttraumatic",'stress','disorder',"traumatic"])]
    
    # Returning a string of all tokenized words joined together with spaces in between.
    return " ".join(text_cleaned)

In [ ]:
# apply function to clean text
df_ptsd['clean_text']= df_ptsd['alltext'].apply(lambda x: clean_text(x)) 

In [ ]:
# Instantiate lemmatizer.
lemmatizer = WordNetLemmatizer()

# define function that takes in words from paragraph, and lemmitize and join back as paragraph
def lemmi_words(text):
    return ' '.join([lemmatizer.lemmatize(i) for i in text.split(' ')])

df_ptsd['lemmi_clean_text'] = df_ptsd['clean_text'].map(lambda x: lemmi_words(x))

In [ ]:
df_export = df_ptsd[['author','subreddit','lemmi_clean_text']]

In [ ]:
# record csv
df_export.to_csv('../Data/TEST_UNSEEN.csv')